In [1]:
import nltk

# 연습문제1

In [3]:
suffix_fdist = nltk.FreqDist()
for word in ['','a','ab','abc']:
    word = word.lower()
    if len(word) == 0:
        continue
    suffix_fdist[word[-1:]] += 1 if len(word) >= 1 else 0
    suffix_fdist[word[-2:]] += 1 if len(word) >= 2 else 0
    suffix_fdist[word[-3:]] += 1 if len(word) >= 3 else 0
suffix_fdist

FreqDist({'a': 1, 'b': 1, 'ab': 1, 'c': 1, 'bc': 1, 'abc': 1})

# 모범답안

In [4]:
suffix_fdist = nltk.FreqDist()
for word in ['','a','ab','abc']:
    word = word.lower()
    if len(word) > 0:
        suffix_fdist[word[-1:]] += 1
    if len(word) > 1:
        suffix_fdist[word[-2:]] += 1
    if len(word) > 2:
        suffix_fdist[word[-3:]] += 1
suffix_fdist

FreqDist({'a': 1, 'b': 1, 'ab': 1, 'c': 1, 'bc': 1, 'abc': 1})

# 연습문제2

In [5]:
from nltk.corpus import brown

suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    if len(word) > 0:
        suffix_fdist[word[-1:]] += 1
    if len(word) > 1:
        suffix_fdist[word[-2:]] += 1
    if len(word) > 2:
        suffix_fdist[word[-3:]] += 1
common_suffixes = [suffix for suffix, count in suffix_fdist.most_common(100)]
common_suffixes[:10]

['e', 's', 'd', 't', 'n', 'he', 'the', 'y', ',', 'r']

In [6]:
def pos_features(word):
    features={}
    for suffix in common_suffixes:
        features[f'endswith({suffix})'] = word.lower().endswith(suffix)
    return features

tagged_words = list(brown.tagged_words(categories='news'))
import random
random.shuffle(tagged_words)
tagged_words[:3]

[('later', 'RBR'), ('down', 'IN'), ('Earl', 'NP')]

In [7]:
featuresets = [(pos_features(n),g) for (n,g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:],featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(classifier,test_set)

0.53078070611636

In [8]:
classifier1 = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier1,test_set)

0.5978120338140229

In [10]:
words = tagged_words[:1000]
featuresets = [(n,pos_features(n),g) for (n,g) in words]
errors_Naive = []
errors_Decision = []
for n,f,g in featuresets:
    naive = classifier.classify(f)
    decision = classifier1.classify(f)
    if naive != g and decision == g:
        errors_Naive.append((n,g,naive,decision))
    if naive == g and decision != g:
        errors_Decision.append((n,g,naive,decision))
print(len(errors_Naive))
print(len(errors_Decision))

98
18


# 모범답안

In [11]:
errors_Naive = []
for (word,tag) in tagged_words[-1000:]:
    guess = classifier.classify(pos_features(word))
    guess1 = classifier1.classify(pos_features(word))
    if guess != tag and guess1 == tag:
        errors_Naive.append((word,tag,guess,guess1))
len(errors_Naive)

92

In [12]:
errors_Decision = []
for (word, tag) in tagged_words[-1000:]:
    guess = classifier.classify(pos_features(word))
    guess1 = classifier1.classify(pos_features(word))
    if guess == tag and guess1 != tag:
        errors_Decision.append((word,tag,guess,guess1))
len(errors_Decision)

21

In [13]:
errors_Decision[:10]

[('nine', 'CD', 'CD', 'NN'),
 ('careful', 'JJ', 'JJ', 'NN'),
 ('leg', 'NN', 'NN', 'JJ'),
 ('has', 'HVZ', 'HVZ', 'CS'),
 ('Kiowa', 'NP', 'NP', 'AT'),
 ('earlier', 'JJR', 'JJR', 'NN'),
 ('it', 'PPO', 'PPO', 'PPS'),
 ('give', 'VB', 'VB', 'JJ'),
 ('said', 'VBD', 'VBD', 'NN'),
 ('Higher', 'JJR', 'JJR', 'AP')]

In [14]:
errors_Naive[:10]

[('he', 'PPS', 'AT', 'PPS'),
 ('at', 'IN', 'CS', 'IN'),
 ('other', 'AP', 'JJR', 'AP'),
 ('with', 'IN', 'ABX', 'IN'),
 ('an', 'AT', 'NP', 'AT'),
 ('She', 'PPS', 'AT', 'PPS'),
 ('sixth-sense', 'NN', 'DTS', 'NN'),
 ('with', 'IN', 'ABX', 'IN'),
 ('line', 'NN', 'CD', 'NN'),
 ('at', 'IN', 'CS', 'IN')]